In [5]:
import requests 

url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
response = requests.get(url)

text = response.text

with open('input.txt', 'w', encoding= 'utf-8') as f:
    f.write(response.text)

In [6]:
print(f"The length of the dataset is : {len(text)}.")

The length of the dataset is : 1115394.


In [10]:
print(text[:300])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us


In [ ]:
chars = sorted(list(set(text))) # all the unique characters
vocab_size = len(chars)

print("".join(chars))
print("Size:", vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
Size: 65


In [14]:
# Tokenization
# Convert the raw text to some sequence of integers 
# Here, characters into integers

stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s] # output is a list
decode = lambda l: ''.join([itos[i] for i in l]) # output is the initial string

print(encode("Hyy there!"))
print(decode(encode("Hyy there")))

[20, 63, 63, 1, 58, 46, 43, 56, 43, 2]
Hyy there


In [15]:
import torch

data = torch.tensor(encode(text), dtype = torch.long)

print(data.shape, data.dtype)
print(data[:300])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [16]:
# split the data into train and validation 

n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [ ]:
block_size = 8 # context length
train_data[:block_size + 1] # it predicts each number one by one

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [ ]:
# we would like the transformer to see everything up to block_size

x = train_data[:block_size]
y = train_data[1:block_size + 1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"When input is {context}, the target is {target}.")

When input is tensor([18]), the target is 47.
When input is tensor([18, 47]), the target is 56.
When input is tensor([18, 47, 56]), the target is 57.
When input is tensor([18, 47, 56, 57]), the target is 58.
When input is tensor([18, 47, 56, 57, 58]), the target is 1.
When input is tensor([18, 47, 56, 57, 58,  1]), the target is 15.
When input is tensor([18, 47, 56, 57, 58,  1, 15]), the target is 47.
When input is tensor([18, 47, 56, 57, 58,  1, 15, 47]), the target is 58.


In [ ]:
torch.manual_seed(42)
batch_size = 4 # how many independent sequences we are processing every pass of the transformer
block_size = 8 # the maximum context length 

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,)) # generate random positions to grab a chunk out of
    x = torch.stack([data[i: i+block_size] for i in ix]) # we stack all the one dimensional tensors that are found at random positions (ROWS)
    y = torch.stack([data[i+1: i+block_size+1] for i in ix])
    return x,y

xb, yb = get_batch('train')
print(f"Inputs: \n {xb.shape}.")
print(xb)
print(f"Targets: \n {yb.shape}.")
print(yb)
print()

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1] # batch b and everything until the time t
        target = yb[b,t]
        print(f"When input is {context.tolist()}, the target is {target}.")

Inputs: 
 torch.Size([4, 8]).
tensor([[57,  1, 46, 47, 57,  1, 50, 53],
        [ 1, 58, 46, 43, 56, 43,  1, 41],
        [17, 26, 15, 17, 10,  0, 32, 53],
        [57, 58,  6,  1, 61, 47, 58, 46]])
Targets: 
 torch.Size([4, 8]).
tensor([[ 1, 46, 47, 57,  1, 50, 53, 60],
        [58, 46, 43, 56, 43,  1, 41, 39],
        [26, 15, 17, 10,  0, 32, 53,  1],
        [58,  6,  1, 61, 47, 58, 46,  0]])

When input is [57], the target is 1.
When input is [57, 1], the target is 46.
When input is [57, 1, 46], the target is 47.
When input is [57, 1, 46, 47], the target is 57.
When input is [57, 1, 46, 47, 57], the target is 1.
When input is [57, 1, 46, 47, 57, 1], the target is 50.
When input is [57, 1, 46, 47, 57, 1, 50], the target is 53.
When input is [57, 1, 46, 47, 57, 1, 50, 53], the target is 60.
When input is [1], the target is 58.
When input is [1, 58], the target is 46.
When input is [1, 58, 46], the target is 43.
When input is [1, 58, 46, 43], the target is 56.
When input is [1, 58, 46

In [31]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# It is a Bigram model that is using the Markov assumption
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets = None):
        logits = self.token_embedding_table(idx) # tensor (batch, time, channel) 

        if targets == None:
            loss = None 
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T)
        for _ in range(max_new_tokens):
            logits, _ = self(idx) # calls the forward method 
            logits = logits[:, -1, :] # (B, C)
            probs = F.softmax(logits, dim = 1)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples = 1) # (B, 1)
            idx = torch.cat((idx, idx_next), dim = 1) # (B, T+1)
        
        return idx
    
model = BigramLanguageModel(vocab_size)
logits, loss = model(xb, yb)
print(logits.shape)
print(loss)

idx = torch.zeros((1,1), dtype = torch.long)
print(decode(model.generate(idx = idx, max_new_tokens = 100)[0].tolist()))

torch.Size([32, 65])
tensor(4.7352, grad_fn=<NllLossBackward0>)

B'jCfNehCzjLtOvZf!VOiO&?,Kt-3PvJGFCvXe?eqEl
t-bz&hMabtWT
;qfCDkBpGKEgToJAuxzqflZrxZ-&jDGmtfx$rPXfE.K
